In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import shutil

drive_path = '/content/drive/MyDrive/su_baseline_artifacts'

local_path = '/content/su_baseline_artifacts'

# Copy the file
shutil.copytree(drive_path, local_path)

print(f"File copied from {drive_path} to {local_path}")

FileExistsError: [Errno 17] File exists: '/content/su_baseline_artifacts'

In [ ]:
!pip install fpdf

In [ ]:
import os
import numpy as np
import joblib
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from report_generator import plot_radar, generate_pdf

# Load data
X = np.load("/content/su_baseline_artifacts/features/X.npy", allow_pickle=True)
y = np.load("/content/su_baseline_artifacts/features/y.npy", allow_pickle=True)
speakers = np.load("speakers.npy", allow_pickle=True)

# Load model (choose either RF or SVM)
MODEL_PATH = "su_baseline_artifacts/models/rf_model.joblib"
model = joblib.load(MODEL_PATH)
model_name = os.path.basename(MODEL_PATH).split(".")[0]

# Emotion to trait mapping
emotion_to_trait_map = {
    'angry':      [0.4, 0.4, 0.5, 0.2, 0.8],
    'happy':      [0.7, 0.5, 0.9, 0.8, 0.3],
    'sad':        [0.3, 0.5, 0.2, 0.4, 0.9],
    'neutral':    [0.5, 0.5, 0.5, 0.5, 0.5],
    'frustrated': [0.3, 0.3, 0.4, 0.3, 0.8],
    'excited':    [0.8, 0.6, 0.9, 0.7, 0.4],
    'fearful':    [0.3, 0.4, 0.3, 0.3, 0.9],
    'disgusted':  [0.4, 0.4, 0.3, 0.2, 0.8],
    'surprised':  [0.7, 0.4, 0.8, 0.6, 0.4],
    'other':      [0.5, 0.5, 0.5, 0.5, 0.5]
}

# Predict
y_pred = model.predict(X)

# Output directory
OUTPUT_DIR = "val_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Generate reports
summary = []

for i, (true_label, pred_label, speaker) in enumerate(zip(y, y_pred, speakers)):
    audio_id = f"sample_{i}"
    true_traits = emotion_to_trait_map.get(true_label, [0.5]*5)
    pred_traits = emotion_to_trait_map.get(pred_label, [0.5]*5)

    # Radar plot
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    labels = ["O", "C", "E", "A", "N"]
    angles = np.linspace(0, 2 * np.pi, len(labels) + 1)

    pred_vals = pred_traits + [pred_traits[0]]
    true_vals = true_traits + [true_traits[0]]

    ax.plot(angles, pred_vals, label="Predicted", color='blue')
    ax.fill(angles, pred_vals, alpha=0.25, color='blue')

    ax.plot(angles, true_vals, label="Actual", color='green')
    ax.fill(angles, true_vals, alpha=0.25, color='green')

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_title(f"Personality Prediction: {audio_id}")
    ax.legend(loc="upper right")

    report_path = os.path.join(OUTPUT_DIR, f"{audio_id}_report.pdf")
    generate_pdf(pred_traits, fig, output_path=report_path)

    summary.append({
        "audio_id": audio_id,
        "speaker": speaker,
        "true_label": true_label,
        "predicted_label": pred_label
    })

# Save CSV
summary_df = pd.DataFrame(summary)
summary_df.to_csv(os.path.join(OUTPUT_DIR, "summary.csv"), index=False)
print("Evaluation complete. Summary CSV saved.")